This notebook was used to scrape professional furniture listing images in an attempt to
circumvent hand labeling AptDeco images as high or low quality.

In [ ]:
import pymysql as mdb
import sys
from bs4 import BeautifulSoup # For HTML parsing
import urllib # Website connections
import re # Regular expressions
from time import sleep # To prevent overwhelming the server between connections
from collections import Counter # Keep track of our term counts
from nltk.corpus import stopwords # Filter out stopwords, such as 'the', 'or', 'and'
import pandas as pd # For converting results to a dataframe and bar chart plots
import math
from string import digits
from functools import reduce
import time
import collections
from http.cookiejar import CookieJar
import ssl
from ImageFeatures import *


# Ashley Furniture

In [ ]:
def get_image_urls(website):
    req=urllib.request.Request(website,data=None,headers={'User-Agent':'Mozilla/5.0 (Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; en-US; rv:1.9.0.5)'})
    cj = CookieJar()
    opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cj))
    opener.addheaders = [('User-Agent', 'Mozilla/5.0 (Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; en-US; rv:1.9.0.5)')]
    opener.addheaders =[('Accept','text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8')]
    response = opener.open(req)
    html = response.read().decode('utf8', errors='ignore')
    response.close()
    
    soup = BeautifulSoup(html,"lxml")
       
    return re.findall(r'"rollOver":{"html":null,"src":"(.*?)"',str(soup))

def get_base_url(base_category,sub_category):
    return 'https://www.ashleyfurniturehomestore.com/c/furniture/%s/%s/' % (base_category,sub_category)

In [ ]:
#BREAK IT DOWN LIKE THIS TO KEEP TRACK OF FURNITURE TYPE

living_room_categories = ['sofas','sectional-sofas','loveseats','living-room-chairs','recliners',\
                         'sleeper-sofas','power-sofas-loveseats-and-recliners','occasional-tables',\
                         'coffee-tables','end-and-side-tables','console-tables','tv-stands-and-media-centers',
                         'ottomans']

kitchen_categories = ['dining-room-tables','chairs','bar-stools','dining-room-sets']

home_office_categories =[''] #listing under one page

entryway_categories = ['']

bedroom_categories = ['']

image_urls=[]

all_types =[]

for category in living_room_categories:
    url = get_base_url('living-room',category)
    image_paths = get_image_urls(url)   
    image_urls += image_paths  
    all_types += ['living-room']*len(image_paths)

    
for category in kitchen_categories:
    url = get_base_url('kitchen-and-dining-room',category)
    image_paths = get_image_urls(url)   
    image_urls += image_paths  
    all_types += ['dining-room']*len(image_paths)


for category in home_office_categories:
    url = get_base_url('home-office',category)
    image_paths = get_image_urls(url)   
    image_urls += image_paths  
    all_types += ['home-office']*len(image_paths)

    
for category in entryway_categories:
    url = get_base_url('entryway',category)
    image_paths = get_image_urls(url)   
    image_urls += image_paths  
    all_types += ['entryway']*len(image_paths)
 
  
for category in bedroom_categories:
    url = get_base_url('bedroom',category)
    image_paths = get_image_urls(url)   
    image_urls += image_paths  
    all_types += ['bedroom']*len(image_paths)
  
    
#Also grab lighting image urls
url = 'https://www.ashleyfurniturehomestore.com/c/lighting/'
image_paths = get_image_urls(url)   
image_urls += image_paths  
all_types += ['lighting']*len(image_paths)
  

In [ ]:
#Scrape it all at once
#furniture_urls = get_image_urls('https://www.ashleyfurniturehomestore.com/c/furniture/?pg=190')
#lighting_urls = get_image_urls('https://www.ashleyfurniturehomestore.com/c/lighting/?pg=100')
#full_cat_urls = get_image_urls('https://www.ashleyfurniturehomestore.com/c/?pg=900')

In [ ]:
df = pd.DataFrame(image_urls,columns=['path'])
df['source']='Ashley'
df['type'] = all_types
df.head()
df.to_csv('ProfessionalImages.csv')

In [ ]:
df.head()

# Furniture.com

In [ ]:
df = pd.read_csv('ProfessionalImages.csv')
df.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
df.tail()

In [ ]:
def get_image_urls(website):
    req=urllib.request.Request(website,data=None,headers={'User-Agent':'Mozilla/5.0 (Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; en-US; rv:1.9.0.5)'})
    cj = CookieJar()
    opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cj))
    opener.addheaders = [('User-Agent', 'Mozilla/5.0 (Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; en-US; rv:1.9.0.5)')]
    opener.addheaders =[('Accept','text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8')]
    response = opener.open(req)
    html = response.read().decode('utf8', errors='ignore')
    response.close()
    
    soup = BeautifulSoup(html,"lxml")
       
    return re.findall(r'"rollOver":{"html":null,"src":"(.*?)"',str(soup))

urls = ['https://www.furniture.com/living-room/seating/sofas',
        'https://www.furniture.com/living-room/seating/chairs',
        'https://www.furniture.com/living-room/seating/loveseats',
        'https://www.furniture.com/living-room/tables',
        'https://www.furniture.com/living-room/seating/ottomans',
        'https://www.furniture.com/living-room/seating/sleeper-sofas',
        'https://www.furniture.com/living-room/seating/sectionals',
        'https://www.furniture.com/living-room/seating/recliners',
        'https://www.furniture.com/dining-room/sets',
        'https://www.furniture.com/dining-room/tables',
        'https://www.furniture.com/dining-room/chairs',
        'https://www.furniture.com/dining-room/bar-stools',
        'https://www.furniture.com/dining-room/china-cabinets-and-curios',
        'https://www.furniture.com/dining-room/buffets-and-sideboards',
        'https://www.furniture.com/dining-room/benches',
        'https://www.furniture.com/bedroom/beds/king',
        'https://www.furniture.com/bedroom/beds/queen',
        'https://www.furniture.com/kids/beds/full',
        'https://www.furniture.com/kids/beds/twin',
        'https://www.furniture.com/living-room/seating/futons',
        'https://www.furniture.com/bedroom/beds/daybeds',
        'https://www.furniture.com/bedroom/dressers',
        'https://www.furniture.com/bedroom/chests',
        'https://www.furniture.com/bedroom/nightstands',
        'https://www.furniture.com/rugs',
        'https://www.furniture.com/home-decor/storage-benches']

all_paths = []
all_types = []
for main in urls:
    website=main
    category = main.split('/')[3]
    
    req=urllib.request.Request(website,data=None,headers={'User-Agent':'Mozilla/5.0 (Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; en-US; rv:1.9.0.5)'})
    cj = CookieJar()
    opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cj))
    opener.addheaders = [('User-Agent', 'Mozilla/5.0 (Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; en-US; rv:1.9.0.5)')]
    opener.addheaders =[('Accept','text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8')]
    response = opener.open(req)
    html = response.read().decode('utf8', errors='ignore')
    response.close()
    
    paths = re.findall(r'<img itemprop="image" data-src="//images.furniture.com(.*?)"',str(html))
    paths = ['images.furniture.com' + path for path in paths]
    all_paths += paths
    all_types += [category]*len(paths)


In [ ]:
temp_df = pd.DataFrame(all_paths,columns=['path'])
temp_df['type'] = all_types
temp_df['source'] = 'Furniture.com'
temp_df.head()
#df = df.append([df,temp_df])

In [ ]:
df = pd.concat([df,temp_df]).reset_index(drop=True)

In [ ]:
df.to_csv('ProfessionalImages.csv')

# Overstock.com

In [ ]:
def get_image_urls(website):
    req=urllib.request.Request(website,data=None,headers={'User-Agent':'Mozilla/5.0 (Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; en-US; rv:1.9.0.5)'})
    cj = CookieJar()
    opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cj))
    opener.addheaders = [('User-Agent', 'Mozilla/5.0 (Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; en-US; rv:1.9.0.5)')]
    opener.addheaders =[('Accept','text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8')]
    response = opener.open(req)
    html = response.read().decode('utf8', errors='ignore')
    response.close()

    paths= re.findall(r'src="https://ak1.ostkcdn.com(.*?)" class="img-responsive product-img"',str(html))
    paths = ['https://ak1.ostkcdn.com' + path for path in paths]
    return paths



In [ ]:
import numpy as np
all_paths = []
all_types = []

#Has 30k living room items
num_images = 10000
num_pages = int(np.floor(num_images/60))
for i in range(0,num_pages-1):
    website = 'https://www.overstock.com/Home-Garden/Living-Room-Furniture/713/cat.html?index=%d&count=60' % (1+60*i)
    paths = get_image_urls(website)
    all_paths += paths
    all_types += ['living-room']*60
    
#18k dining room
num_images = 10000
num_pages = int(np.floor(num_images/60))
for i in range(0,num_pages-1):
    website = 'https://www.overstock.com/Home-Garden/Dining-Room-Bar-Furniture/711/cat.html?index=%d&count=60' % (1+60*i)
    paths = get_image_urls(website)
    all_paths += paths
    all_types += ['dining-room']*60
       
#16k bedroom
num_images = 10000
num_pages = int(np.floor(num_images/60))
for i in range(0,num_pages-1):
    website = 'https://www.overstock.com/Home-Garden/Bedroom-Furniture/710/cat.html?index=%d&count=60' % (1+60*i)
    paths = get_image_urls(website)
    all_paths += paths
    all_types += ['bedroom']*60

# 8k bathroom
num_images = 8000
num_pages = int(np.floor(num_images/60))
for i in range(0,num_pages-1):
    website = 'https://www.overstock.com/Home-Garden/Bathroom-Furniture/1454/cat.html?index=%d&count=60' % (1+60*i)
    paths = get_image_urls(website)
    all_paths += paths
    all_types += ['bathroom']*60



In [ ]:
temp_df = pd.DataFrame(all_paths,columns=['path'])
temp_df['type'] = all_types
temp_df['source'] = 'Overstock.com'
temp_df.head()

In [ ]:
len(temp_df)

In [ ]:
df = pd.concat([df,temp_df]).reset_index(drop=True)

In [ ]:
df.to_csv('ProfessionalImages.csv')

# Scrape image features

In [ ]:
df = pd.read_csv('ProfessionalImages.csv')
df.drop('Unnamed: 0',axis=1,inplace=True)
df = df.sample(frac=1,random_state=102).reset_index(drop=True)
df.ix[df['source']=='Furniture.com','path']='http://'+df.ix[df['source']=='Furniture.com']['path']

In [ ]:
df = pd.read_csv('ProfessionalImageFeatures.csv')
df.drop('Unnamed: 0',axis=1,inplace=True)


In [ ]:
len(df.dropna())

In [ ]:
for idx in range(len(df.dropna()),len(df)):
    try:
        #Get Image
        image = url_to_image(df.ix[idx,'path'])
        
        #Remove White borders
        white_pixels = (image == 255).all(axis=2)
        frac_white = len(white_pixels[white_pixels==True])/white_pixels.size

        #if frac_white > 40%, don't crop, since it is background subtracted
        if frac_white < 0.40:
            img_g = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
            goodndx_row = np.where(np.var(image,axis=1)>2)[0]
            goodndx_col = np.where(np.var(image,axis=0)>10)[0]
            if len(image.shape)>2:
                img2 = image[goodndx_row,:,:]
                image = img2[:,goodndx_col,:]
            else:
                img2 = image[goodndx_row,:]
                image = img2[:,goodndx_col]
        
        #Extract features
        features=ExtractFeatures(image)

        #Populate df
        for key in features.keys():
            df.loc[idx,key] = features[key]

        #Save every 2000 images process (~30 minutes)
        if (idx % 2000 == 0):
            df.to_csv('ProfessionalImageFeatures.csv')
    except:
        print("Problem with grabbing image")

In [ ]:
df.to_csv('ProfessionalImageFeatures.csv')